In [1]:
# librerias
from statsmodels.tsa.stattools import pacf
from statsmodels.tsa.stattools import acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.stattools import adfuller

from pmdarima.arima import auto_arima
from pmdarima.arima import ARIMA

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

In [3]:
metano_df = pd.read_csv('contaminantes.csv', header=0, index_col=0)
print(len(metano_df))
metano_df.head()

45204


,NO2 (µg/m³),CH (mg/m3),not CH4 (mg/m3),SP (µg/m³),NO (µg/m³),O3 (µg/m³),Toluene (µg/m³),Benzene (µg/m³)
Fecha,,,,,,,,
2016-01-01T00:00:00.000Z,50.0,7.5,0.1,40.0,25.0,2.0,1.8,0.7
2016-01-01T01:00:00.000Z,54.0,9.4,0.1,52.0,40.0,2.0,3.0,0.8
2016-01-01T02:00:00.000Z,58.0,8.4,0.2,76.0,62.0,2.0,2.8,1.1
2016-01-01T03:00:00.000Z,48.0,6.7,0.1,55.0,14.0,4.0,1.3,0.7
2016-01-01T04:00:00.000Z,49.0,7.0,0.1,31.0,22.0,1.0,0.5,0.5


In [4]:
metano_df.dropna(inplace=True)

In [5]:
print(metano_df.columns)
len(metano_df)

Index(['NO2 (µg/m³)', 'CH (mg/m3)', 'not CH4 (mg/m3)', 'SP (µg/m³)',
       'NO (µg/m³)', 'O3 (µg/m³)', 'Toluene (µg/m³)', 'Benzene (µg/m³)'],
      dtype='object')


35131

## Modelo ARIMA para una columna como target

In [6]:
def model_arima(df, column_name= None, train_size= 0.8, start_p= 1, start_q= 1, max_p=5, max_q= 5, max_d= 3): 
    """ 
    Function description:
    The function trains an ARIMA model for a given Time Series.
    We split train and test using the parameter train_size (from 0 to 1 where 0.5 is 50%)
    Then we call the auto-arima function to get the best parameters
    Finally we use those best params to train our ARIMA model
    
    ARIMA Model:
    We can combine both Autoregressive (AR) and Moving Average (MA) models together to create an ARMA model. The time series is regressed on the previous values and the previous errors.
    For ARMA(p, q), p is the order of the AR process and q is the order of the MA process.

    Parameters:
    df (Pandas DataFrame type): is the given dataframe

    column_name (string): the chosen column to create new columns from

    train_size (float): value from 0 to 1 where 0.5 is 50% representing the % of our data 
    that will be used to train the mdoel

    start_p (int): initial value for p parameter. By default 1
    
    start_q (int): initial value for q parameter. By default 1

    max_p (int): max value for p parameter. By default 5

    max_q (int): max value for q parameter. By default 5

    max_d (int): max value for d parameter. By default 3

    return: our trained ARIMA model
    """
    df_lenght = len(df)
    train_cut_point = int(train_size * df_lenght)

    train = df[column_name][:train_cut_point]

    model_aa = auto_arima(train,
                        start_p = start_p,
                        start_q = start_q,
                        max_p = max_p,
                        max_q = max_q,
                        max_d = max_d,
                        trace=True)

    best_order = model_aa.order

    model = ARIMA(order= best_order)
    model_for_pickle = model.fit(train)
    
    return model_for_pickle

In [ ]:
#predictions = model.predict(n_periods= len(test))
#print("mean_squared_error:", mean_squared_error(test, predictions))

#### Llamamos a la funcion del modelo

In [7]:
modelo = model_arima(metano_df, 'NO2 (µg/m³)')

Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=215909.146, Time=6.17 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=217994.031, Time=0.86 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=216348.901, Time=0.89 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=215935.421, Time=5.90 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=217992.032, Time=0.51 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=212816.623, Time=18.34 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=215593.116, Time=3.85 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=212805.209, Time=40.28 sec
 ARIMA(3,1,0)(0,0,0)[0] intercept   : AIC=215337.322, Time=5.92 sec
 ARIMA(4,1,1)(0,0,0)[0] intercept   : AIC=212782.351, Time=65.14 sec
 ARIMA(4,1,0)(0,0,0)[0] intercept   : AIC=215014.654, Time=8.95 sec
 ARIMA(5,1,1)(0,0,0)[0] intercept   : AIC=212780.448, Time=81.08 sec
 ARIMA(5,1,0)(0,0,0)[0] intercept   : AIC=214691.504, Time=7.79 sec
 ARIMA(5,1,2)(0,0,0)[0] intercept   : AIC=212782.556, Time=63.33 sec


####  Guardamos el modelo

In [8]:
# Guardar el modelo
import pickle

with open('finished_model_arima.model', "wb") as archivo_salida:
    pickle.dump(modelo, archivo_salida)

#### Abrimos el modelo para comprobar que se ha guardado correctamente

In [9]:
# Para volver a leer el modelo
with open('finished_model∫_arima.model', "rb") as archivo_entrada:
    modelo_arima = pickle.load(archivo_entrada)
    
print(modelo_arima)

 ARIMA(5,1,1)(0,0,0)[0] intercept


#### Predecimos

In [10]:
modelo.predict(10)

array([13.23029504, 13.48920768, 13.69252769, 13.83391184, 13.92356939,
       13.97227635, 13.99229046, 13.9950331 , 13.9890949 , 13.98004709])

#### Calculando los errores

In [11]:
test = metano_df['NO2 (µg/m³)'][10001:12000]
predictions = modelo.predict(n_periods= len(test))
print("mean_squared_error:", mean_squared_error(test, predictions))
print("mean_absolute_error:", mean_absolute_error(test, predictions))

mean_squared_error: 649.0310017032789
mean_absolute_error: 16.388388191314302


## Modelo ARIMA con bucle for para varias columnas

In [12]:
def model_arima_multiple(df, train_size= 0.8, start_p= 1, start_q= 1, max_p=5, max_q= 5, max_d=3):
    """ 
    Function description:
    The function trains an ARIMA model for a given Time Series with multiple variables
    The function will train an ARIMA model for each given variable in a dataframe
    We split train and test using the parameter train_size (from 0 to 1 where 0.5 is 50%)
    Then we call the auto-arima function to get the best parameters
    Finally we use those best params to train our ARIMA model
    
    ARIMA Model:
    We can combine both Autoregressive (AR) and Moving Average (MA) models together to create an ARMA model. The time series is regressed on the previous values and the previous errors.
    For ARMA(p, q), p is the order of the AR process and q is the order of the MA process.

    Parameters:
    df (Pandas DataFrame type): is the given dataframe

    column_name (string): the chosen column to create new columns from

    train_size (float): value from 0 to 1 where 0.5 is 50% representing the % of our data 
    that will be used to train the mdoel

    start_p (int): initial value for p parameter. By default 1
    
    start_q (int): initial value for q parameter. By default 1

    max_p (int): max value for p parameter. By default 5

    max_q (int): max value for q parameter. By default 5

    max_d (int): max value for d parameter. By default 3

    return: our trained ARIMA model
    """
    list_columns = list(df.columns)
    list_models = []
    for i in list_columns:
        df_lenght = len(df)
        train_cut_point = int(train_size * df_lenght)
        train = df[i][:train_cut_point]

        model = auto_arima(train,
                        start_p = start_p,
                        start_q = start_q,
                        max_p = max_p,
                        max_q = max_q,
                        max_d = max_d,
                        trace=True
                        )

        best_order = model.order

        model = ARIMA(order= best_order)
        model_for_pickle = model.fit(train)
        list_models.append(model_for_pickle)
    
    return list_models

In [13]:
list_models = model_arima_multiple(metano_df, train_size= 0.8, start_p= 1, start_q= 1, max_p=2, max_q= 2, max_d=2)

Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=215909.146, Time=7.32 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=217994.031, Time=1.09 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=216348.901, Time=1.12 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=215935.421, Time=6.76 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=217992.032, Time=0.89 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=212816.623, Time=22.89 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=215593.116, Time=6.35 sec
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=93.84 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=213058.863, Time=48.85 sec
 ARIMA(2,1,1)(0,0,0)[0]             : AIC=212814.627, Time=7.68 sec
 ARIMA(1,1,1)(0,0,0)[0]             : AIC=215907.146, Time=4.25 sec
 ARIMA(2,1,0)(0,0,0)[0]             : AIC=215591.116, Time=1.58 sec
 ARIMA(2,1,2)(0,0,0)[0]             : AIC=inf, Time=26.22 sec
 ARIMA(1,1,0)(0,0,0)[0]             : AIC=216346.901, Time=0.76 sec
 ARIMA(1,1,2)(0

In [14]:
list_models

[ARIMA(order=(2, 1, 1)),
 ARIMA(order=(1, 1, 2)),
 ARIMA(order=(2, 1, 2)),
 ARIMA(order=(2, 1, 2)),
 ARIMA(order=(2, 1, 2)),
 ARIMA(order=(2, 1, 2)),
 ARIMA(order=(2, 1, 2)),
 ARIMA(order=(2, 1, 2))]

#### Guardamos el pickle del modelo multiple

In [15]:
# Guardar el modelo
import pickle

with open('finished_model_arima_multiple.model', "wb") as archivo_salida:
    pickle.dump(list_models, archivo_salida)

#### Comprobamos que funciona

In [16]:
# Para volver a leer el modelo
with open('finished_model_arima_multiple.model', "rb") as archivo_entrada:
    list_models = pickle.load(archivo_entrada)
    
print(list_models)

[ARIMA(order=(2, 1, 1)), ARIMA(order=(1, 1, 2)), ARIMA(order=(2, 1, 2)), ARIMA(order=(2, 1, 2)), ARIMA(order=(2, 1, 2)), ARIMA(order=(2, 1, 2)), ARIMA(order=(2, 1, 2)), ARIMA(order=(2, 1, 2))]


#### Predicciones para las 9 variables

In [18]:
pred_list = []
for i in list_models:
    predictions = i.predict(n_periods= 2)
    pred_list.append(predictions)

pred_list

[array([13.27465542, 13.58311355]),
 array([1.38929802, 1.39687785]),
 array([0.09996029, 0.09992312]),
 array([26.60162879, 26.23467323]),
 array([0.9972544 , 0.99432561]),
 array([76.12042916, 76.26562191]),
 array([1.6088282 , 1.57856887]),
 array([0.4245239 , 0.43747536])]